<a href="https://colab.research.google.com/github/ndtuan10/CS431_L21_KHCL/blob/main/BaiTap/CNN_MobileNetV2%2BFood_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BÀI TẬP 4**
# **Ngô Đức Tuấn**
MSSV: 18520186

- CNN, Model: MobileNetV2, Dataset: Food-11

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import các thư viện cần dùng
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
print('''
  Phiên bản của các thư viện :
    + Tensorflow : {0}
    + keras : {1}
    + numpy : {2}
    + pandas : {3}
'''.format(tf.__version__, keras.__version__, np.__version__, pd.__version__))


  Phiên bản của các thư viện :
    + Tensorflow : 2.4.1
    + keras : 2.4.0
    + numpy : 1.19.5
    + pandas : 1.1.5



#**1.Giới thiệu về Dataset Food-11**
 Dataset Food-11 bao gồm 16.643 ảnh màu về food được nhóm thành 11 *lớp*. Toàn bộ dataset được chia thành 3 phần như sau: training, validation và evaluation. Dataset có tổng kích thước tệp là 1,16GB.

 *Nhãn* là một mảng của các số nguyên từ 0 đến 10, tương ứng với mỗi *lớp* được định nghĩa theo bảng sau:

<table>
  <tr>
    <th>Nhãn</th><th>0</th><th>1</th><th>2</th><th>3</th><th>4</th><th>5</th><th>6</th><th>7</th><th>8</th><th>9</th><th>10</th>
    
  </tr>
  <tr>
    <td>Loại</td>
    <td>Bánh mì</td>
    <td>Thực phẩm từ sữa</td>
    <td>Tráng miệng</td>
    <td>Trứng</td>
    <td>Món chiên xào</td>
    <td>Thịt</td>
    <td>Mì/Pasta</td>
    <td>Cơm</td>
    <td>Hải sản/td>
    <td>Soup</td>
    <td>Rau củ/Quả</td>
  </tr>
</table>

 Ta sẽ tải dataset về tại đây [Food-11](grebvm2.epfl.ch/lin/food/Food-11.zip)  hoặc load dataset trong drive theo đường dẫn bên dưới.



## **2. Xử lí dataset**

In [3]:
Path_dir = '/content/drive/MyDrive/CS331.L22.KHCL/Dataset/Food-11/img/'
path_train = Path_dir + 'training/'
path_test = Path_dir + 'evaluation/'
path_valid = Path_dir + 'validation/'

In [4]:
classNames = os.listdir(path_test)
print(classNames)

['Dessert', 'Bread', 'Seafood', 'Meat', 'Soup', 'Fried food', 'Dairy product', 'Rice', 'Noodles-Pasta', 'Egg', 'Vegetable-Fruit']


In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

In [6]:
def loadData(path):
  A, B = [], []
  for folder_name in classNames:
    folder_path = os.path.join(Path_dir, path, folder_name)
    i = 0
    for fname in os.listdir(folder_path):
      if (i<700):
        fpath = os.path.join(folder_path, fname)
        print(fpath)
        img = image.load_img(fpath, target_size=(224,224))
        img_data = image.img_to_array(img)
        A.append(img_data)
        B.append(folder_name)
        i = i+1
  # Preprocessing
  A = np.asarray(A)/255.0
  B = np.asarray(B)
  print('Loading completed. ')
  return A,B

In [7]:
# load tập train, test, validation
A_train, B_train = loadData(path_train)
A_test, B_test = loadData(path_test)
A_valid, B_valid = loadData(path_valid)

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/CS331.L22.KHCL/Dataset/Food-11/img/evaluation/Meat/249.jpg
/content/drive/MyDrive/CS331.L22.KHCL/Dataset/Food-11/img/evaluation/Meat/229.jpg
/content/drive/MyDrive/CS331.L22.KHCL/Dataset/Food-11/img/evaluation/Meat/351.jpg
/content/drive/MyDrive/CS331.L22.KHCL/Dataset/Food-11/img/evaluation/Meat/360.jpg
/content/drive/MyDrive/CS331.L22.KHCL/Dataset/Food-11/img/evaluation/Meat/369.jpg
/content/drive/MyDrive/CS331.L22.KHCL/Dataset/Food-11/img/evaluation/Meat/168.jpg
/content/drive/MyDrive/CS331.L22.KHCL/Dataset/Food-11/img/evaluation/Meat/53.jpg
/content/drive/MyDrive/CS331.L22.KHCL/Dataset/Food-11/img/evaluation/Meat/341.jpg
/content/drive/MyDrive/CS331.L22.KHCL/Dataset/Food-11/img/evaluation/Meat/246.jpg
/content/drive/MyDrive/CS331.L22.KHCL/Dataset/Food-11/img/evaluation/Meat/419.jpg
/content/drive/MyDrive/CS331.L22.KHCL/Dataset/Food-11/img/evaluation/Meat/44.jpg
/content/drive/MyDrive/CS331.L22.KHCL/Dataset/Foo

In [8]:
# Khai phá dữ liệu 
print('A_train: {}, B_train: {}, A_valid: {}, B_valid: {}, A_test: {}, B_test: {}'.format(A_train.shape, B_train.shape, A_valid.shape, B_valid.shape, A_test.shape, B_test.shape))

A_train: (6749, 224, 224, 3), B_train: (6749,), A_valid: (3148, 224, 224, 3), B_valid: (3148,), A_test: (3347, 224, 224, 3), B_test: (3347,)


In [9]:
result = {
    'Data':["A_train", "B_train", "A_valid", "B_valid", "A_test", "B_test"],
    'Shape': [A_train.shape, B_train.shape, A_valid.shape, B_valid.shape, A_test.shape, B_test.shape]
}
pd.DataFrame(result)

,Data,Shape
0,A_train,"(6749, 224, 224, 3)"
1,B_train,"(6749,)"
2,A_valid,"(3148, 224, 224, 3)"
3,B_valid,"(3148,)"
4,A_test,"(3347, 224, 224, 3)"
5,B_test,"(3347,)"


# **3. Tiền xử lý dữ liệu**

In [10]:
# One hot encoding
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
B_train = lb.fit_transform(B_train)
B_valid = lb.fit_transform(B_valid)

# **4. Huấn luyện mô hình mạng CNN**

## **MobileNetV2**

### **1. Thiết kế kiến trúc cho mạng**

In [11]:
# Xây dựng kiến trúc mạng
model = Sequential()

mobileV2 = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),include_top=False, weights='imagenet')
mobileV2.trainable = False

model.add(mobileV2)
model.add(GlobalAveragePooling2D())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(len(classNames)))
model.add(Activation('softmax'))

# in ra kiến trúc của mô hình
model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               655872    
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 11)                5643      
___________________________________________________________

### **2. Biên dịch mô hình**
Trước khi mô hình có thể được huấn luyện, chúng ta cần thêm các hàm toán học giúp cho việc đánh giá hay làm mô hình đi " đúng hướng ". Các chỉnh sửa này được thêm vào trong bước *biên dịch* của mô hình:

* *Hàm mất mát* — dùng để tính toán "độ sai sót" của mô hình trong quá trình huấn luyện và dự đoán. Chúng ta cần phải làm cho nghiệm của hàm này nhỏ nhất để kiểm soát mô hình đi đúng hướng. (mất mát càng ít, chính xác càng cao). *Hàm mất mát* hay *loss function* được sử dụng là `crossentropy`.
* *Hàm tối ưu* — Đây là "công cụ" để làm hàm mất mát có nghiệm nhỏ nhất, qua việc cập nhật và chỉnh sửa các tham số có trong hàm mất mát. *Hàm tối ưu* hay *optimizier function* được sử dụng là `Adam`.
* *Thang đo* — dùng để đo đạc độ chính xác qua mỗi quá trình huấn luyện và kiểm tra. Chúng ta sẽ sử dụng phép đo `accuracy`, nhằm nắm được tỉ lệ ảnh được phân loại chính xác.

In [12]:
# Biên dịch mô hình
aug = ImageDataGenerator(rotation_range=0.18,zoom_range=0.15,width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
learning_rate = 0.01
epochs = 30
batch_size = 64
opt = Adam(learning_rate=learning_rate)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])

### **3. Train mô hình**

In [13]:
# đưa dữ liệu vào và huấn luyện
print('Start training')
import time
start=time.time()
H = model.fit(aug.flow(A_train,B_train,batch_size=batch_size), validation_data=(A_valid,B_valid), steps_per_epoch=A_train.shape[0]//batch_size, epochs=epochs, verbose=1)
stop = time.time()
mobilenetV2_time = {stop - start}
print(f"Training time: {mobilenetV2_time}s")

Start training
Epoch 1/30
105/105 [==============================] - 105s 677ms/step - loss: 1.2244 - accuracy: 0.6478 - val_loss: 1.0387 - val_accuracy: 0.7240
Epoch 2/30
105/105 [==============================] - 68s 643ms/step - loss: 0.6190 - accuracy: 0.7984 - val_loss: 0.8937 - val_accuracy: 0.7363
Epoch 3/30
105/105 [==============================] - 68s 646ms/step - loss: 0.5848 - accuracy: 0.8048 - val_loss: 0.8355 - val_accuracy: 0.7478
Epoch 4/30
105/105 [==============================] - 68s 645ms/step - loss: 0.5337 - accuracy: 0.8228 - val_loss: 0.7179 - val_accuracy: 0.7706
Epoch 5/30
105/105 [==============================] - 68s 643ms/step - loss: 0.4560 - accuracy: 0.8415 - val_loss: 0.7768 - val_accuracy: 0.7583
Epoch 6/30
105/105 [==============================] - 68s 644ms/step - loss: 0.4594 - accuracy: 0.8412 - val_loss: 0.9782 - val_accuracy: 0.7271
Epoch 7/30
105/105 [==============================] - 68s 646ms/step - loss: 0.4017 - accuracy: 0.8655 - val_loss:

In [14]:
# lưu mô hình xuống file
model.save('MobilenetV2.h5')

### **4.Đánh giá mô hình**

In [15]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [16]:
# đưa ra dự đoán ảnh
from sklearn.metrics import confusion_matrix, classification_report

label_test = lb.fit_transform(B_test)

B_pred = model.predict(A_test)
print(B_pred.shape)

B_pred = np.argmax(B_pred, axis = 1)
label_test = np.argmax(label_test, axis = 1)

B_pred = B_pred.astype('str')
label_test = label_test.astype('str')

for i in range(B_pred.shape[0]):
  B_pred[i] = classNames[int(B_pred[i])]
  label_test[i] = classNames[int(label_test[i])]

print('Confusion Matrix')
confusion_matrix(label_test, B_pred)

(3347, 11)
Confusion Matrix


array([[100,   0,   0,   2,   3,   0,   2,   0,  30,   8,   3],
       [  0, 145,   0,   0,   0,   0,   0,   1,   1,   0,   0],
       [  1,   1, 274,   3,   4,  14,   8,   0,  23,  38,   2],
       [  1,   0,   0, 476,   0,   3,   4,   2,  13,   1,   0],
       [  0,   0,   7,   3, 339,   4,   8,   1,  41,  26,   3],
       [  8,   1,  34,   1,   6, 229,   5,   0,  32,  13,   6],
       [  5,   1,   4,   1,   1,   3, 268,   0,  12,   1,   7],
       [  0,   1,   1,   2,   0,   1,   0,  84,   3,   2,   2],
       [ 12,   1,  16,  11,   9,   6,  21,   3, 399,  13,   9],
       [  1,   0,  11,   5,   4,   2,   3,   0,   9, 248,   4],
       [  1,   2,   0,   0,   0,   1,   5,   1,   2,   1, 218]])

In [17]:
print('Classification Report')
print(classification_report(label_test, B_pred, target_names=classNames))

Classification Report
                 precision    recall  f1-score   support

        Dessert       0.78      0.68      0.72       148
          Bread       0.95      0.99      0.97       147
        Seafood       0.79      0.74      0.77       368
           Meat       0.94      0.95      0.95       500
           Soup       0.93      0.78      0.85       432
     Fried food       0.87      0.68      0.77       335
  Dairy product       0.83      0.88      0.85       303
           Rice       0.91      0.88      0.89        96
  Noodles-Pasta       0.71      0.80      0.75       500
            Egg       0.71      0.86      0.78       287
Vegetable-Fruit       0.86      0.94      0.90       231

       accuracy                           0.83      3347
      macro avg       0.84      0.84      0.84      3347
   weighted avg       0.84      0.83      0.83      3347



In [18]:
accuracy_score_MobileNetV2 = accuracy_score(label_test, B_pred)
print(f'Do chinh xac (Accuracy) MobileNetV2: {np.multiply(metrics.accuracy_score(label_test, B_pred),100)} %')

Do chinh xac (Accuracy) MobileNetV2: 83.05945622945922 %


In [20]:
results_dict={'Phuong phap phan lop':['MobileNetV2'],
              'Accuracy':['83.06%']}
pd.DataFrame(results_dict)

,Phuong phap phan lop,Accuracy
0,MobileNetV2,83.06%
